In [1]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)

import time

import torch
# import numpy as np
import pandas as pd
# import torchvision as tv
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import (
    LearningRateMonitor,
    ModelCheckpoint,
    EarlyStopping,
)
from src.utils.helpers import load_config
from src.training.dataset import ImageDataModule
from src.models.classification_model import ImageClassifier

C:\Users\camme\anaconda3\envs\DL_FinalProject\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\camme\anaconda3\envs\DL_FinalProject\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
real_filepaths = os.listdir("../../src/data/sampled/train/real")
fake_filepaths = os.listdir("../../src/data/sampled/train/ai")

real_count, fake_count = len(real_filepaths), len(fake_filepaths)
print(f"REAL COUNT: {real_count} ({round(real_count/(fake_count+real_count) * 100, 1)}%)")
print(f"FAKE COUNT: {fake_count} ({round(fake_count/(fake_count+real_count) * 100, 1)}%)")

REAL COUNT: 16800 (50.0%)
FAKE COUNT: 16800 (50.0%)


In [3]:
torch.set_float32_matmul_precision("high")
config = load_config("sampled_convnext_tiny.yaml")
pl.seed_everything(config.get("seed"))

csv_logger = CSVLogger(
    save_dir=config["logging"].get("logs_dir", "logs"),
    name=config.get("experiment_name"),
)

experiment_name = config.get("experiment_name")

Seed set to 7643


In [4]:
callbacks = []
data_module = ImageDataModule(config)

# Initialize model:
net = ImageClassifier(config)

# Enable learning rate monitoring hook:
if config["training"].get("lr_monitoring"):
    monitor_lr = LearningRateMonitor(logging_interval="epoch")
    callbacks.append(monitor_lr)

# Enable early stopping hook:
estop = config["training"].get("early_stopping", {})
if estop.get("enabled"):
    early_stopping = EarlyStopping(
        monitor=estop.get("monitor", "val_loss"),
        patience=estop.get("patience", 5),
        mode=estop.get("mode", "min"),
        verbose=True,
    )
    callbacks.append(early_stopping)

# Enable model checkpointing hook:
checkpoint = ModelCheckpoint(
    dirpath=config["logging"].get("checkpoint_dir", "checkpoints"),
    filename="{epoch:02d}-{val_acc:.2f}",
    save_top_k=1,
    monitor="val_acc",
    mode="max",
    save_last=True,
)
callbacks.append(checkpoint)

In [5]:
# Configure training session:
trainer = pl.Trainer(
    min_epochs=config["training"].get("min_epochs"),
    max_epochs=config["training"].get("max_epochs", -1),
    logger=csv_logger,
    enable_checkpointing=True,
    enable_progress_bar=True,
    enable_model_summary=True,
    devices="auto",
    accelerator="auto",
    log_every_n_steps=5,
    callbacks=callbacks,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
start = time.time()
# Begin training:
trainer.fit(
    model=net,
    datamodule=data_module,
)
end = time.time()
print(f"Took {end - start:.3f} seconds")

[NOTICE] Found 0 corrupted files in ../../src/data/scaled-bicubic/train.
[NOTICE] Found 0 corrupted files in ../../src/data/scaled-bicubic/validation.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | loss_func | CrossEntropyLoss | 0      | train
1 | backbone  | ConvNeXt         | 27.8 M | train
-------------------------------------------------------
27.8 M    Trainable params
0         Non-trainable params
27.8 M    Total params
111.287   Total estimated model params size (MB)
250       Modules in train mode
0         Modules in eval mode


[NOTICE] Found 0 corrupted files in ../../src/data/scaled-bicubic/test.
Epoch 0: 100%|██████████| 350/350 [02:56<00:00,  1.99it/s, v_num=0]        
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 350/350 [03:05<00:00,  1.89it/s, v_num=0, val_loss=0.122, val_acc=0.952, train_loss=0.219, train_acc=0.902]

Metric val_loss improved. New best score: 0.122


Epoch 1: 100%|██████████| 350/350 [02:55<00:00,  2.00it/s, v_num=0, val_loss=0.122, val_acc=0.952, train_loss=0.219, train_acc=0.902]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 350/350 [03:03<00:00,  1.90it/s, v_num=0, val_loss=0.0741, val_acc=0.974, train_loss=0.0476, train_acc=0.983]

Metric val_loss improved by 0.047 >= min_delta = 0.0. New best score: 0.074


Epoch 2: 100%|██████████| 350/350 [02:55<00:00,  2.00it/s, v_num=0, val_loss=0.0741, val_acc=0.974, train_loss=0.0476, train_acc=0.983]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 350/350 [02:55<00:00,  2.00it/s, v_num=0, val_loss=0.0813, val_acc=0.975, train_loss=0.0151, train_acc=0.995]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 350/350 [02:55<00:00,  2.00it/s, v_num=0, val_loss=0.113, val_acc=0.971, train_loss=0.00902, train_acc=0.997]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 350/350 [03:04<00:00,  1.90it/s, v_num=0, val_loss=0.0963, val_acc=0.975, train_loss=0.00571, train_acc=0.998]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.074. Signaling Trainer to stop.
`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 350/350 [03:04<00:00,  1.89it/s, v_num=0, val_loss=0.0963, val_acc=0.975, train_loss=0.00571, train_acc=0.998]
Took 985.259 seconds


In [7]:
# Test best model:
trainer.test(
    model=net,
    datamodule=data_module,
    ckpt_path="checkpoints/last.ckpt",
)

[NOTICE] Found 0 corrupted files in ../../src/data/scaled-bicubic/train.
[NOTICE] Found 0 corrupted files in ../../src/data/scaled-bicubic/validation.


Restoring states from the checkpoint path at checkpoints/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at checkpoints/last.ckpt


[NOTICE] Found 0 corrupted files in ../../src/data/scaled-bicubic/test.
Testing DataLoader 0: 100%|██████████| 75/75 [00:07<00:00,  9.51it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9757896065711975
        test_loss           0.08740879595279694
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.08740879595279694, 'test_acc': 0.9757896065711975}]

In [8]:
# Review training metrics:
training_metrics_df = pd.read_csv(
     "logs/" + experiment_name + "/version_1/metrics.csv"
)
training_metrics_df

FileNotFoundError: [Errno 2] No such file or directory: 'logs/sampled_convnext_tiny_scaledbicubic_jt/version_1/metrics.csv'

In [ ]:
def plot_metrics(filepath: str):
    metrics_df = pd.read_csv(filepath)
    train_acc = metrics_df["train_acc"].dropna(ignore_index=True)
    val_acc = metrics_df["val_acc"].dropna(ignore_index=True)
    train_loss = metrics_df["train_loss"].dropna(ignore_index=True)
    val_loss = metrics_df["val_loss"].dropna(ignore_index=True)

    # Losses:
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
    ax1.plot(train_loss, label="Train Loss", c="skyblue")
    ax1.plot(val_loss, label="Validation Loss", c="red")
    # Labels:
    ax1.set_title("Loss Curves")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.set_xticks(range(0, len(train_loss)))
    ax1.grid()
    ax1.legend()
    # Accuracies:
    ax2.plot(train_acc, label="Train Accuracy", c="green")
    ax2.plot(val_acc, label="Validation Accuracy", c="purple")
    # Labels:
    ax2.set_title("Accuracy Curves")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy")
    ax2.set_xticks(range(0, len(train_acc)))
    ax2.grid()
    ax2.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_metrics("logs/sampled_convnext_tiny_scaled_jt/version_1/metrics.csv")